In [1]:
import pandas as pd

In [2]:
month_convert = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril", 5: "Maio", 6: "Junho",
    7: "Julho", 8: "Agosto", 9: "Setembro", 10: "Outubro", 11: "Novemrbo", 12: "Dezembro"
}

# Análise do CSV _pessoas_completo_

In [3]:
df = pd.read_csv("../data/pessoas_completo.csv")

In [4]:
df.head()

,id_sinistro,municipio,tipo_via,tipo_veiculo_vitima,sexo,idade,data_obito,gravidade_lesao,tipo_de vítima,faixa_etaria_demografica,...,profissao,data_sinistro,ano_sinistro,mes_sinistro,dia_sinistro,ano_mes_sinistro,ano_obito,mes_obito,dia_obito,ano_mes_obito
0,2463323,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,51.0,01/01/2015,FATAL,PEDESTRE,50 a 54,...,NaN,01/01/2015,2015,1,1,2015/01,2015.0,1.0,1.0,2015/01
1,2465127,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,89.0,03/01/2015,FATAL,PEDESTRE,85 a 89,...,NaN,03/01/2015,2015,1,3,2015/01,2015.0,1.0,3.0,2015/01
2,2462123,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,67.0,05/01/2015,FATAL,PEDESTRE,65 a 69,...,NaN,05/01/2015,2015,1,5,2015/01,2015.0,1.0,5.0,2015/01
3,2506869,MOGI GUACU,VIAS MUNICIPAIS,NAO DISPONIVEL,NAO DISPONIVEL,44.0,09/01/2015,FATAL,NAO DISPONIVEL,40 a 44,...,NaN,09/01/2015,2015,1,9,2015/01,2015.0,1.0,9.0,2015/01
4,2490194,COTIA,RODOVIAS,AUTOMOVEL,MASCULINO,28.0,11/01/2015,FATAL,CONDUTOR,25 a 29,...,NaN,11/01/2015,2015,1,11,2015/01,2015.0,1.0,11.0,2015/01


In [5]:
df.count()

id_sinistro                 1030387
municipio                   1030380
tipo_via                    1030387
tipo_veiculo_vitima           58157
sexo                        1030387
idade                       1013700
data_obito                    58157
gravidade_lesao             1030387
tipo_de vítima               992638
faixa_etaria_demografica    1030387
faixa_etaria_legal          1030387
profissao                    852894
data_sinistro               1030387
ano_sinistro                1030387
mes_sinistro                1030387
dia_sinistro                1030387
ano_mes_sinistro            1030387
ano_obito                     58157
mes_obito                     58157
dia_obito                     58157
ano_mes_obito                 58157
dtype: int64

In [6]:
df["faixa_etaria_legal"].value_counts()

18-24             255252
25-29             151949
30-34             119826
35-39             105705
40-44              89428
45-49              67832
0-17               57686
50-54              52031
55-59              38860
60-64              27391
65-69              18895
NAO DISPONIVEL     16687
70-74              12662
75-79               8355
80 ou mais          7828
Name: faixa_etaria_legal, dtype: int64

## Removendo colunas e linhas
Alguns dados da tabela estão com valores desnecessários pra análise

Verificando se todos os acidentes que são fatais contém data do óbito

In [7]:
((df["gravidade_lesao"] == "FATAL") & (df["data_obito"].isna())).sum()

0

Removendo colunas **ano_mes_obito, ano_mes_sinistro, data_obito** e **data_sinistro** pois são redundantes

In [8]:
df = df.drop(columns=["ano_mes_sinistro", "ano_mes_obito", "data_obito", "data_sinistro"])

Removendo linhas com valor nulo nas colunas **municipio** e **tipo_via** 

In [9]:
df = df.dropna(subset=["municipio", "tipo_via"])

Verificando se **faixa_etaria_demografica** e **faixa_etaria_legal** são redundantes

In [10]:
print("faixa demográfica:", df["faixa_etaria_demografica"].unique())
print("faixa legal:", df["faixa_etaria_legal"].unique())

faixa demográfica: ['50 a 54' '85 a 89' '65 a 69' '40 a 44' '25 a 29' '60 a 64' '05 a 09'
 '30 a 34' '20 a 24' '15 a 19' '00 a 04' '70 a 74' '75 a 79'
 'NAO DISPONIVEL' '35 a 39' '55 a 59' '45 a 49' '80 a 84' '10 a 14'
 '90 e +']
faixa legal: ['50-54' '80 ou mais' '65-69' '40-44' '25-29' '60-64' '0-17' '30-34'
 '18-24' '70-74' '75-79' 'NAO DISPONIVEL' '35-39' '55-59' '45-49']


Como os valores de ambas colunas são fortemente semelhantes optarei por manter apenas uma delas

In [11]:
df = df.drop("faixa_etaria_demografica", axis=1)

## Renomeação e conversão de colunas e valores de linhas
Muitos dados precisam ser tratados para poderem ser analisados mais apropriadamente

Renomeando coluna **tipo_de vítima** para **tipo_vitima**

In [12]:
df = df.rename(columns={"tipo_de vítima": "tipo_vitima"})

Adicionando idade média nos valores nulos da coluna **idade** e renomeando valores nulos das colunas **profissao** e **tipo_vitima**

In [13]:
df["idade"] = df["idade"].fillna(df["idade"].mean())
df["profissao"] = df["profissao"].fillna("DESCONHECIDA")
df["tipo_vitima"] = df["tipo_vitima"].fillna("NAO DISPONIVEL")

Renomeando valores _nao disponivel_ da coluna **sexo** para _ignorado_

In [14]:
df["sexo"] = df["sexo"].replace({"NAO DISPONIVEL": "IGNORADO"})

Mudando tipo de dado das colunas **idade, mes_obito, mes_sinistro, ano_obito** e **dia_obito** de `float64` para `int64`

In [15]:
df["idade"] = df["idade"].round().astype("Int64")
df["ano_obito"] = df["ano_obito"].astype("Int64")
df["dia_obito"] = df["dia_obito"].astype("Int64")

mudando valores mensais das colunas **mes_sinistro** e **mes_obito** de numérico para texto

In [16]:
df["mes_obito"] = df["mes_obito"].astype("Int64").map(month_convert)
df["mes_sinistro"] = df["mes_sinistro"].astype("Int64").map(month_convert)

## Salvando versão limpa em CSV

In [17]:
df.to_csv("../data/pessoas_completo_limpo.csv", index=False)

# Analise do CSV _sinistros_completo_ 

In [18]:
df = pd.read_csv("../data/sinistros_completo.csv")

In [19]:
df[["conservacao", "administracao", "jurisdicao", "tipo_acidente_primario", "regiao_administrativa"]].head()

,conservacao,administracao,jurisdicao,tipo_acidente_primario,regiao_administrativa
0,PREFEITURA,PREFEITURA,MUNICIPAL,ATROPELAMENTO,Campinas
1,PREFEITURA,PREFEITURA,MUNICIPAL,NAO DISPONIVEL,Barretos
2,PREFEITURA,PREFEITURA,MUNICIPAL,ATROPELAMENTO,Metropolitana de São Paulo
3,PREFEITURA,PREFEITURA,MUNICIPAL,ATROPELAMENTO,Metropolitana de São Paulo
4,PREFEITURA,PREFEITURA,MUNICIPAL,NAO DISPONIVEL,Baixada Santista


In [20]:
df.isna().sum()

id_sinistro                              0
tipo_registro                            0
data_sinistro                            0
ano_sinistro                             0
mes_sinistro                             0
dia_sinistro                             0
ano_mes_sinistro                         0
hora_sinistro                         3403
logradouro                            3203
numero_logradouro                     9550
tipo_via                                 0
latitude                              6901
longitude                             6900
municipio                                7
regiao_administrativa                    7
tp_veiculo_bicicleta               1210186
tp_veiculo_caminhao                1190700
tp_veiculo_motocicleta              757437
tp_veiculo_nao_disponivel          1097217
tp_veiculo_onibus                  1210370
tp_veiculo_outros                  1213214
tp_veiculo_automovel                728174
gravidade_nao_disponivel           1131782
gravidade_l

In [21]:
cols_tp_veiculo = [col for col in df.columns if col.startswith("tp_veiculo")]
total = df[cols_tp_veiculo].count()
total.sum()

1279303

Pela soma de tipo de veiculos por sinistro ser maior que o número total de sinistros podemas analisar que há acidentes com diferentes tipos de veículos e por isso não daria pra juntar todas as colunas **tp_veiculo_xxx** em uma só, e isso tambem ocorre para as colunas **tp_sinistro**

## Removendo colunas

In [22]:
df = df.drop(columns=["latitude", "longitude", "data_sinistro", "ano_mes_sinistro"])

## Renomeando valores

Mudando todos os valores nulos das colunas **tp_sinistro_xxx** para "N" e os das colunas **tp_veiculos_xxx, raviade_xxx** para 0 (int)

In [23]:
colunas_binarias = [col for col in df.columns if col.startswith("tp_sinistro")]
df[colunas_binarias] = df[colunas_binarias].fillna("N")
colunas_binarias = [col for col in df.columns if col.startswith("gravidade_") or col.startswith("tp_veiculo_")]
df[colunas_binarias] = df[colunas_binarias].fillna(0)

Mudando os tipos de dados das colunas **tp_veiculo_xxx**, **gravidade_xxx** e **numero_logradouro** de `float64` para `int64`

In [24]:
df[colunas_binarias] = df[colunas_binarias].astype(int)
df["numero_logradouro"] = df["numero_logradouro"].round().astype("Int64")

Mudando os valores dos meses de numérico para texto

In [25]:
df["mes_sinistro"] = df["mes_sinistro"].astype("Int64").map(month_convert)

Transformando os valores de texto para inteiro

In [26]:
df["hora_sinistro"] = pd.to_datetime(df["hora_sinistro"], format="%H:%M", errors="coerce").dt.hour
df["hora_sinistro"] = df["hora_sinistro"].astype("Int64")

## Salvando versão limpa em CSV

In [27]:
df.to_csv("../data/sinistros_completo_limpo.csv", index=False)

# Analise do CSV _veiculos_completo_ 

In [28]:
df = pd.read_csv("../data/veiculos_completo.csv")

In [29]:
df.head()

,id_sinistro,ano_fab,ano_modelo,cor_veiculo,data_sinistro,ano_sinistro,mes_sinistro,dia_sinistro,ano_mes_sinistro,tipo_veiculo
0,2501575,NaN,NaN,NaN,21/12/2014,2014,12,21,2014/12,OUTROS
1,2456933,NaN,NaN,NaN,23/12/2014,2014,12,23,2014/12,OUTROS
2,2456933,1994.0,1995.0,Azul,23/12/2014,2014,12,23,2014/12,AUTOMOVEL
3,2456933,2011.0,2011.0,Prata,23/12/2014,2014,12,23,2014/12,AUTOMOVEL
4,2463759,2008.0,2008.0,Prata,26/12/2014,2014,12,26,2014/12,AUTOMOVEL


## Removendo colunas

In [30]:
df = df.drop(columns=["data_sinistro", "ano_mes_sinistro"])

## Renomeando valores

Trocando valores da coluna **mes_sinistro** de numérico para texto, e mudando os tipos de valores de **ano_fab** e **ano_modelo** de `float64` para `Int64`

In [31]:
df["mes_sinistro"] = df["mes_sinistro"].astype("Int64").map(month_convert)
df["ano_fab"] = df["ano_fab"].astype("Int64")
df["ano_modelo"] = df["ano_modelo"].astype("Int64")

In [32]:
df["ano_fab"].value_counts()

0       177061
2011     74557
2019     68270
2013     65554
2018     63500
         ...  
75           1
85           1
15           1
5015         1
2812         1
Name: ano_fab, Length: 175, dtype: Int64

Colocando a moda como valor no lugar dos valores nulos das colunas **ano_fab** e **ano_modelo**

In [33]:
moda = df.loc[df["ano_fab"] != 0, "ano_fab"].mode()
df["ano_fab"] = df["ano_fab"].fillna(moda)
moda = df.loc[df["ano_modelo"] != 0, "ano_modelo"].mode()
df["ano_modelo"] = df["ano_modelo"].fillna(moda)

Renomeando os valores nulos da coluna **cor_veiculo**

In [34]:
df["cor_veiculo"] = df["cor_veiculo"].fillna("IGNORADO")

## Salvando versão limpa em CSV

In [35]:
df.to_csv("../data/veiculos_completo_limpo.csv", index=False)